# Transfer Learning and Fine Tuning

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

#### Using GPU (highly recommended)

-> If using `theano` backend:

`THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32`

In [1]:
import numpy as np
import datetime

np.random.seed(1337)  # for reproducibility

In [18]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Conv2D
from keras.utils import np_utils
from keras import backend as K
from numpy import nan

now = datetime.datetime.now

### Settings

In [3]:
now = datetime.datetime.now

batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [8]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)
K.image_data_format()

'channels_last'

In [36]:
input_shape

(28, 28, 1)

In [54]:
def train_model(model, train, test, nb_classes):
    print train[0].shape
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    print X_train[0].shape
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(X_train, Y_train,
              batch_size=batch_size, epochs=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

## Dataset Preparation

In [64]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
print y_train[y_train >= 5][0:10]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
print y_train_gte5[:10]
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

[5 9 5 6 7 8 6 9 9 7]
[0 4 0 1 2 3 1 4 4 2]


In [90]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (X_train_lt5, y_train_lt5),
            (X_test_lt5, y_test_lt5), nb_classes)

(30596, 28, 28)
(28, 28, 1)
('X_train shape:', (30596, 28, 28, 1))
(30596, 'train samples')
(5139, 'test samples')
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
24448/30596 [======================>.......] - ETA: 2s - loss: 0.1452 - acc: 0.9566

KeyboardInterrupt: 

In [22]:
nb_classes

5

In [60]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (X_train_gte5, y_train_gte5),
            (X_test_gte5, y_test_gte5), nb_classes)

(29404, 28, 28)
(28, 28, 1)
('X_train shape:', (29404, 28, 28, 1))
(29404, 'train samples')
(4861, 'test samples')
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 13s - loss: 0.3995 - acc: 0.8812 - val_loss: 0.0992 - val_acc: 0.9685
Epoch 2/5
29404/29404 [==============================] - 13s - loss: 0.1296 - acc: 0.9611 - val_loss: 0.0607 - val_acc: 0.9798
Epoch 3/5
29404/29404 [==============================] - 14s - loss: 0.0909 - acc: 0.9714 - val_loss: 0.0449 - val_acc: 0.9860
Epoch 4/5
29404/29404 [==============================] - 14s - loss: 0.0757 - acc: 0.9773 - val_loss: 0.0406 - val_acc: 0.9868
Epoch 5/5
29404/29404 [==============================] - 16s - loss: 0.0666 - acc: 0.9799 - val_loss: 0.0357 - val_acc: 0.9885
Training time: 0:01:12.759073
('Test score:', 0.035686860284623395)
('Test accuracy:', 0.9884797365693393)


In [86]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (X_train_gte5, y_train_gte5),
            (X_test_gte5, y_test_gte5), nb_classes)

(29404, 28, 28)
(28, 28, 1)
('X_train shape:', (29404, 28, 28, 1))
(29404, 'train samples')
(4861, 'test samples')
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
 7552/29404 [======>.......................] - ETA: 10s - loss: 0.0584 - acc: 0.9823

KeyboardInterrupt: 

In [85]:
np.vstack((X_test_gte5, X_test_lt5))

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ..., 
       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, 

In [94]:
X_test = X_test_lt5.reshape((X_test_lt5.shape[0],) + input_shape)
X_test = X_test.astype('float32')
X_test /= 255
    
y_old = model.predict(X_test)

In [95]:
y_old = [np.argmax(v) for v in y_old]

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_lt5, y_old)

0.99299474605954463

In [84]:
X_test_gte5.shape

(4861, 28, 28)

In [83]:
np.vstack((X_test_gte5, X_test_lt5)).shape

(10000, 28, 28)

## Your Turn

Try to Fine Tune a VGG16 Network

In [10]:
## your code here

```python
...
...
# Plugging new Layers
    model.add(Dense(768, activation='sigmoid'))
    model.add(Dropout(0.0))
    model.add(Dense(768, activation='sigmoid'))
    model.add(Dropout(0.0))
    model.add(Dense(n_labels, activation='softmax'))
```

In [59]:
(1,) + (3, 4)

(1, 3, 4)